### Youtube Sentence Adjust Word List Ratio

In [1]:
import pandas as pd
import numpy as np
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import glob

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [2]:
folder_name = "Turkish"
word_num = 28

#### Calculate Sentence Ratio By Word

In [3]:
df_adjust_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/Level 1/Opus/Word_{word_num}.xlsx")
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [4]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(40000)
df_youtube_sentence.reset_index(inplace=True)
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM
...,...,...,...,...
387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE
387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE
387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE
387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE


In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(df_adjust_word["word"].to_list())
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = (len(intersect_word)/len(sent_word)+0.001)*100
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [ ]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

In [8]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"kimsin, peki",sen
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"parolayı, söyle",NaN
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"kim, ya, olacağım",ben
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"ım, keloğlan",NaN
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,ey,NaN
...,...,...,...,...,...,...,...,...
387903,387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE,16.766667,"çıkmadı, bile, an, aklımdan, gözlerin",bir
387904,387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE,0.100000,"bin, hala, lira, o, ufak, koynumdaysa",NaN
387905,387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE,0.100000,sahi,NaN
387906,387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE,20.100000,"hala, sende, eşi, diğer",mi


In [9]:
#df_sentence_ratio_result.to_excel(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.xlsx", index=False)

#### Calculate Videoid Ratio By Word

In [13]:
#df_adjust_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/Level 1/Opus/Word_{word_num}.xlsx")
#df_adjust_word

In [14]:
#df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(5000)
#df_youtube_sentence

In [15]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence

,video_id,sentence
0,23c9nwRm01o,arkadaşlar merhaba uzun zamandır video çekmiyo...
1,3jq75SKokNQ,asi döner alabilir miyim buyrun ne kadar hızlı...
2,4Ncsac3ywv0,intro çalıyor aman of of of of gidiyom gidemiy...
3,8V9tq1pe8eI,özgür bunlar normalde kamyon daha büyük araçla...
4,96Fw6H0ZG84,hiçbir şey yapmayacağım damlalarını vereceğim ...
5,CU7tcdhf02I,kar yağar kar üstüne derdim var dert üstüne ce...
6,DKTAB8HgIwc,dam başında sarı çiçek oy oy dam başında sarı ...
7,DgtKLrT9_GI,behey ala gözlü dilber dilber vaktin geçer dem...
8,DjrAJH4bjpE,eser rüzgar dağıtmaz efkarımı ben yarından umu...
9,EttVyq4DHYk,ve keten tohumu jeli ve kremi yapıyoruz sarkma...


In [16]:
def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
    word_set = set(df_word[f"{word_column_name}"].to_list())
    for i in range(len(df_videoid_sentence)):
        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
        text_word = re.findall(r"\w+", text, re.UNICODE)
        text_word_set = set(text_word)
        intersect_word = list(word_set.intersection(text_word_set))
        different_word = list(text_word_set.difference(word_set))
        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = (len(intersect_word)/len(text_word)+0.001)*100
        df_videoid_sentence.loc[i,"different_word"] = ", ".join(different_word)
        df_videoid_sentence.loc[i,"intersect_word"] = ", ".join(intersect_word)
        
    return df_videoid_sentence

In [17]:
df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", 28)
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,23c9nwRm01o,arkadaşlar merhaba uzun zamandır video çekmiyo...,6.160606,"ozaman, biraz, uzun, bütün, ile, panjur, anlam...","bir, var, bu, ama, ve, çok"
1,3jq75SKokNQ,asi döner alabilir miyim buyrun ne kadar hızlı...,5.933333,"afiyet, pisliği, hızlısı, hemen, nasıl, sakin,...","de, bir, bu, ben, var, ne, kadar"
2,4Ncsac3ywv0,intro çalıyor aman of of of of gidiyom gidemiy...,4.751163,"gidemiyom, gitar, anasına, kalmaz, aman, çalıy...","sen, bu, ben, ama, bana, ve"
3,8V9tq1pe8eI,özgür bunlar normalde kamyon daha büyük araçla...,1.562317,"içerde, kurt, sigaranla, karın, sıcak, durmuşu...","sen, şey, evet, yok, bana, var, bunu, daha, be..."
4,96Fw6H0ZG84,hiçbir şey yapmayacağım damlalarını vereceğim ...,3.983495,"hissediyorum, havluyu, sorunları, karar, üzeri...","sen, şey, yok, bana, var, bunu, daha, beni, bi..."
5,CU7tcdhf02I,kar yağar kar üstüne derdim var dert üstüne ce...,4.645455,"zaman, yağar, halım, arpa, sevmem, gelin, cell...","var, seni"
6,DKTAB8HgIwc,dam başında sarı çiçek oy oy dam başında sarı ...,1.549275,"başında, düşte, ürgüp, burdan, sarı, gör, kese...","bir, de"
7,DgtKLrT9_GI,behey ala gözlü dilber dilber vaktin geçer dem...,0.637634,"keser, harami, geçer, dilber, lerin, gözlü, be...",mi
8,DjrAJH4bjpE,eser rüzgar dağıtmaz efkarımı ben yarından umu...,3.325806,"yazık, baharımı, çaldılar, kuşlar, verip, böyl...","ben, mı, ne, için"
9,EttVyq4DHYk,ve keten tohumu jeli ve kremi yapıyoruz sarkma...,3.098236,"jelimizin, çatlaklara, yağı, sıcak, arta, sıvı...","daha, iyi, ve, için, bir, de, yok, bu, ben, da..."


In [18]:
#df_text_ratio_result.to_excel(f"Youtube_Videoid_Text_{word_num}_Word_Ratio_Result.xlsx", index=False)

#### Select Adjust Sentence Ratio Result

In [19]:
#df_sentence_ratio_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name}/Deployment/Result/Level 1/Youtube/Youtube_Sentence_28_Word_Ratio_Result.xlsx")
#df_sentence_ratio_result

In [20]:
adjust_word_ratio = 100
df_adjust_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_word_ratio]
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
59,59,00:00:15.780,00:00:16.600,ne,3jq75SKokNQ,100.1,NaN,ne
162,162,00:00:20.980,00:00:21.480,var,tCadoXCM31Y,100.1,NaN,var
164,164,00:00:23.280,00:00:24.060,var,tCadoXCM31Y,100.1,NaN,var
342,342,00:04:18.495,00:04:19.495,yok,UDDfBEUGY60,100.1,NaN,yok
350,350,00:04:45.747,00:04:46.747,ne,UDDfBEUGY60,100.1,NaN,ne
351,351,00:04:47.900,00:04:48.900,ne,UDDfBEUGY60,100.1,NaN,ne
400,400,00:07:35.246,00:07:36.278,değil mi evet,UDDfBEUGY60,100.1,NaN,"değil, mi, evet"
419,419,00:09:15.573,00:09:17.263,ben de çok,UDDfBEUGY60,100.1,NaN,"ben, de, çok"
549,549,00:21:32.301,00:21:33.545,sen de,UDDfBEUGY60,100.1,NaN,"sen, de"
564,564,00:23:33.434,00:23:34.434,evet,UDDfBEUGY60,100.1,NaN,evet


In [21]:
#df_adjust_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Sentences.xlsx", index=False)

#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [22]:
df_adjust_ratio["with_word"] = 1
df_adjust_ratio

<ipython-input-22-3eb37270623c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_ratio["with_word"] = 1


,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
59,59,00:00:15.780,00:00:16.600,ne,3jq75SKokNQ,100.1,NaN,ne,1
162,162,00:00:20.980,00:00:21.480,var,tCadoXCM31Y,100.1,NaN,var,1
164,164,00:00:23.280,00:00:24.060,var,tCadoXCM31Y,100.1,NaN,var,1
342,342,00:04:18.495,00:04:19.495,yok,UDDfBEUGY60,100.1,NaN,yok,1
350,350,00:04:45.747,00:04:46.747,ne,UDDfBEUGY60,100.1,NaN,ne,1
351,351,00:04:47.900,00:04:48.900,ne,UDDfBEUGY60,100.1,NaN,ne,1
400,400,00:07:35.246,00:07:36.278,değil mi evet,UDDfBEUGY60,100.1,NaN,"değil, mi, evet",1
419,419,00:09:15.573,00:09:17.263,ben de çok,UDDfBEUGY60,100.1,NaN,"ben, de, çok",1
549,549,00:21:32.301,00:21:33.545,sen de,UDDfBEUGY60,100.1,NaN,"sen, de",1
564,564,00:23:33.434,00:23:34.434,evet,UDDfBEUGY60,100.1,NaN,evet,1


In [23]:
df_adjust_ratio_flag = pd.merge(df_youtube_sentence, df_adjust_ratio, how="left", on=["index","sentence","start_time","end_time","video_id",f"{word_num}_word_ratio","different_word","intersect_word"])
df_adjust_ratio_flag["with_word"].fillna(0, inplace=True)
df_adjust_ratio_flag

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"kimsin, peki",sen,0.0
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"parolayı, söyle",NaN,0.0
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"ya, olacağım, kim",ben,0.0
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"keloğlan, ım",NaN,0.0
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,ey,NaN,0.0
...,...,...,...,...,...,...,...,...,...
4995,4995,00:03:26.780,00:03:28.160,böyle bile yiyebiliriz,cQjX9DzXTw0,0.100000,"böyle, yiyebiliriz, bile",NaN,0.0
4996,4996,00:03:28.260,00:03:30.540,kıvama geldiğinde artık tamamdır,cQjX9DzXTw0,0.100000,"kıvama, tamamdır, artık, geldiğinde",NaN,0.0
4997,4997,00:03:30.540,00:03:34.840,böyle ağzınıza böyle çıtır çıtır şeker tadı da...,cQjX9DzXTw0,7.792308,"önemli, tadı, lazım, çıtır, gelmemesi, ağzınız...",da,0.0
4998,4998,00:03:34.840,00:03:37.060,şimdi bundan sonra artık çırpıcı ile devam ede...,cQjX9DzXTw0,0.100000,"sonra, bundan, şimdi, artık, devam, ile, edece...",NaN,0.0


In [24]:
df_adjust_ratio_flag.with_word.value_counts()

0.0    4948
1.0      52
Name: with_word, dtype: int64

In [25]:
# Code Running For Video_id Sentence
zeros = []
ones = []
zero_list = []
one_list = []
for i in range(len(df_adjust_ratio_flag)):
    videoid = df_adjust_ratio_flag.loc[i,"video_id"]    
    if df_adjust_ratio_flag.loc[i,"with_word"] == 0:
        zeros.append(i)
        try:
            one_list.append((ones[0],len(ones)))
        except:
            pass
        ones = []
    else:
        ones.append(i)
        try:
            zero_list.append((zeros[0],len(zeros)))
        except:
            pass
        zeros = []

    try:
        if df_adjust_ratio_flag.loc[i+1,"video_id"] != videoid:
            try:
                one_list.append((ones[0],len(ones)))
                ones = []
            except:
                pass
            try:
                zero_list.append((zeros[0],len(zeros)))
                zeros = []
            except:
                pass
        else:
            pass
    except:
        pass

try:
    one_list.append((ones[0],len(ones)))
except:
    pass
try:
    zero_list.append((zeros[0],len(zeros)))
except:
    pass

In [26]:
df_one = pd.DataFrame(one_list)
df_one.rename(columns={0:"index",1:"length"}, inplace=True)
df_one.drop_duplicates(inplace=True)
df_one.sort_values(by="length", ascending=False, inplace=True)
df_one.reset_index(drop=True, inplace=True)
df_one

,index,length
0,350,2
1,3371,2
2,59,1
3,3398,1
4,1735,1
5,2338,1
6,2566,1
7,2580,1
8,2641,1
9,3153,1


In [27]:
df_one_all = pd.DataFrame()
for i in range(len(df_one)):
    index = df_one.loc[i,"index"]
    length = df_one.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_all = pd.concat([df_one_all, df_video],axis=0)
df_one_all.reset_index(inplace=True, drop=True)
df_one_all

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,350,00:04:45.747,00:04:46.747,ne,UDDfBEUGY60,100.1,NaN,ne,1.0
1,351,00:04:47.900,00:04:48.900,ne,UDDfBEUGY60,100.1,NaN,ne,1.0
2,3371,00:05:10.989,00:05:12.100,hayır,tUNXI8BQGGg,100.1,NaN,hayır,1.0
3,3372,00:05:12.145,00:05:12.791,hayır,tUNXI8BQGGg,100.1,NaN,hayır,1.0
4,59,00:00:15.780,00:00:16.600,ne,3jq75SKokNQ,100.1,NaN,ne,1.0
5,3398,00:06:20.754,00:06:21.418,şey,tUNXI8BQGGg,100.1,NaN,şey,1.0
6,1735,00:01:00.400,00:01:02.800,seni,Rpz3oiCcgSg,100.1,NaN,seni,1.0
7,2338,00:16:19.520,00:16:21.560,ama,GWsDgpOaPH8,100.1,NaN,ama,1.0
8,2566,00:29:17.180,00:29:19.700,ben de,GWsDgpOaPH8,100.1,NaN,"ben, de",1.0
9,2580,00:30:05.560,00:30:06.800,bu ne,GWsDgpOaPH8,100.1,NaN,"bu, ne",1.0


In [28]:
#df_one_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Sequential_Sentences_And_Others.xlsx", index=False)

In [29]:
# Only Sequential Sentences
df_one_sequential = df_one[df_one.length > 1]
df_one_sequential

,index,length
0,350,2
1,3371,2


In [30]:
df_one_sequential_all = pd.DataFrame()
for i in range(len(df_one_sequential)):
    index = df_one_sequential.loc[i,"index"]
    length = df_one_sequential.loc[i,"length"]
    start_index = index
    end_index = index+length
    df_video = df_adjust_ratio_flag.iloc[start_index:end_index,]
    df_one_sequential_all = pd.concat([df_one_sequential_all, df_video],axis=0)
df_one_sequential_all.reset_index(inplace=True, drop=True)
df_one_sequential_all

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word,with_word
0,350,00:04:45.747,00:04:46.747,ne,UDDfBEUGY60,100.1,NaN,ne,1.0
1,351,00:04:47.900,00:04:48.900,ne,UDDfBEUGY60,100.1,NaN,ne,1.0
2,3371,00:05:10.989,00:05:12.100,hayır,tUNXI8BQGGg,100.1,NaN,hayır,1.0
3,3372,00:05:12.145,00:05:12.791,hayır,tUNXI8BQGGg,100.1,NaN,hayır,1.0


In [31]:
#df_one_sequential_all.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Only_Sequential_Sentences.xlsx", index=False)

#### Count Sentence Words By Adjust Column

In [32]:
#df_one_sequential_all = pd.read_excel("Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Only_Sequential_Sentences.xlsx")
#df_one_sequential_all

In [33]:
def text_to_list_df(df_target, target_column): # df_target include dict value columns, target_column occurs dict
    text_value_list = []
    for i in df_target[f"{target_column}"]:
        try:
            var = re.findall(r"\w+", i)
            for j in var:
                text_value_list.append(j)
        except:
            pass
    df_alone_value = pd.DataFrame(text_value_list)
    df_alone_value.rename(columns={0:f"{target_column}_alone"}, inplace=True)
    df_unique_count = pd.DataFrame(df_alone_value.value_counts(ascending=False))
    df_unique_count.reset_index(inplace=True)
    df_unique_count.rename(columns={0:"count"}, inplace=True)
            
    return df_unique_count

In [34]:
df_unique_word_count = text_to_list_df(df_one_sequential_all, "intersect_word")
df_unique_word_count

,intersect_word_alone,count
0,hayır,2
1,ne,2


In [35]:
#df_unique_word_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Sequential_Sentence_Words_Count.xlsx", index=False)

#### Count Adjust Sentence Ratio Result By Videoid

In [36]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")["28_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={"28_word_ratio":"28_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by="28_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

,video_id,28_word_ratio_count
0,UDDfBEUGY60,26
1,tUNXI8BQGGg,6
2,EttVyq4DHYk,5
3,cm40e1ChozE,5
4,GWsDgpOaPH8,4
5,tCadoXCM31Y,2
6,3jq75SKokNQ,1
7,8V9tq1pe8eI,1
8,Rpz3oiCcgSg,1
9,ZcTP1fmOFFI,1


In [37]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Videoid_Count.xlsx", index=False)

#### Multi Processing

Number of CPU cores: 16


,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"peki, kimsin",sen
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"parolayı, söyle",
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"ya, kim, olacağım",ben
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"ım, keloğlan",
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,ey,
...,...,...,...,...,...,...,...,...
387903,387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE,16.766667,"an, bile, gözlerin, çıkmadı, aklımdan",bir
387904,387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE,0.100000,"o, bin, lira, ufak, koynumdaysa, hala",
387905,387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE,0.100000,sahi,
387906,387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE,20.100000,"sende, eşi, hala, diğer",mi


,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"peki, kimsin",sen
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"parolayı, söyle",NaN
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"ya, kim, olacağım",ben
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"ım, keloğlan",NaN
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,ey,NaN
...,...,...,...,...,...,...,...,...
387903,387903,00:02:11.720,00:02:14.440,gözlerin bir an bile çıkmadı aklımdan,7R3K6EDaBGE,16.766667,"an, bile, gözlerin, çıkmadı, aklımdan",bir
387904,387904,00:02:15.560,00:02:19.320,koynumdaysa hala o ufak bin lira,7R3K6EDaBGE,0.100000,"o, bin, lira, ufak, koynumdaysa, hala",NaN
387905,387905,00:02:20.320,00:02:21.200,sahi,7R3K6EDaBGE,0.100000,sahi,NaN
387906,387906,00:02:21.720,00:02:24.480,diğer eşi sende mi hala,7R3K6EDaBGE,20.100000,"sende, eşi, hala, diğer",mi
